Project for DISCERN 2024
===

This project is supposed to use [gpt2 model from huggingface](https://huggingface.co/openai-community/gpt2) and then use train data from [kaggle](https://www.kaggle.com/datasets/emineyetm/fake-news-detection-datasets) to detect fake and true information in news and articles. The articles used for checking collected by our team [are here](https://unirau-my.sharepoint.com/:x:/g/personal/dovhan_o_nikita22_stud_rau_ro/EVZaoVJ1OIFFmkT7ognXzbcBiR8JDDXK-ID0DWAdiFnMvg?e=DTESAz). The discussion took its place on WhatsApp and the ideas were stored in [the notebook](https://unirau-my.sharepoint.com/:o:/g/personal/dovhan_o_nikita22_stud_rau_ro/Eu7uK8WA5nZMl_BVm5tbJ2QB0kGIaLwCmCnDAKRrof4tSg?e=edO5it)

Main inspiration taken from the [folder of this programme](https://unirau-my.sharepoint.com/personal/andrei_luchici_rau_ro/_layouts/15/onedrive.aspx?ga=1&id=%2Fpersonal%2Fandrei%5Fluchici%5Frau%5Fro%2FDocuments%2F24%2D02%2D05%20%2D%20DISCERN%20%2D%20Ed%201)

Participants:
---
* Nikita Dovhan
* Zita Kósz
* Daniela Dumitrică
* konstantina Bakopoulou
* Issa El Kaouri
* ChatGPT 

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import GPT2Tokenizer, TFGPT2Model, TFGPT2ForSequenceClassification
import tensorflow as tf
import re

Parameters
---

In [ ]:
#true_file = "/content/drive/MyDrive/mount/True.csv"
#fake_file = "/content/drive/MyDrive/mount/Fake.csv"
true_file = "data/True.csv"
fake_file = "data/Fake.csv"

model_name = "gpt2"
model_dir = "results/the_model_of_the_new_day"

amount_of_entries = 5000
batch_size = 8

In [ ]:
true_df = pd.read_csv(true_file)
fake_df = pd.read_csv(fake_file)

In [ ]:
true_df["label"] = 1
fake_df["label"] = 0

In [ ]:
combined_df = pd.concat([true_df, fake_df], ignore_index=True)
combined_df = combined_df.sample(frac=1, random_state=42).reset_index(drop=True)
combined_df = combined_df.head(amount_of_entries)

In [ ]:
combined_df['text'] = combined_df['text'].apply(lambda x: re.sub(r'^[A-Z\s]+\([A-Za-z\s]+\)\s*-\s*', '', x))

In [ ]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    combined_df["text"].values,
    combined_df["label"].values,
    test_size=0.2,
    random_state=42
)

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=512, return_tensors='tf')
test_encodings = tokenizer(test_texts.tolist(), truncation=True, padding=True, max_length=512, return_tensors='tf')

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
))

In [ ]:
class GPT2Classifier(tf.keras.Model):
    def __init__(self, num_classes, l2_lambda=0.01):
        super(GPT2Classifier, self).__init__()
        self.gpt2 = TFGPT2Model.from_pretrained(model_name)
        self.dropout = tf.keras.layers.Dropout(0.3)
        self.dense = tf.keras.layers.Dense(num_classes, activation='softmax',
                                           kernel_regularizer=tf.keras.regularizers.L2(l2_lambda))

    def call(self, inputs):
        outputs = self.gpt2(inputs)[0]
        pooled_output = tf.reduce_mean(outputs, axis=1)
        pooled_output = self.dropout(pooled_output)
        logits = self.dense(pooled_output)
        return logits

model = GPT2Classifier(num_classes=2)
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True 
)

In [ ]:
model.fit(train_dataset.shuffle(amount_of_entries).batch(batch_size),
          epochs=3,
          batch_size=batch_size,
          callbacks=[early_stopping],
          validation_data=test_dataset.batch(batch_size))  

In [ ]:
loss, accuracy = model.evaluate(test_dataset.batch(batch_size))
print(f"Test accuracy: {accuracy}")

In [ ]:
model.save(model_dir)

Testing
===

In [ ]:
def predict_news_class(probabilities, threshold=0.5):
    fake_probability, true_probability = probabilities[0]
    if fake_probability > threshold:
        return "fake"
    else:
        return "true"

In [ ]:
model = tf.keras.models.load_model(model_dir)

In [ ]:
inputs = tokenizer(input(), return_tensors='tf', max_length=512, truncation=True, padding='max_length')
output = model(inputs)
predict_news_class(output)